In [1]:
# run stopwatch
from tools import Stopwatch
stopwatch = Stopwatch()
start = stopwatch.start()

### Load libraries, functions, palette, theme

In [2]:
%run _libraries.ipynb

In [3]:
%run _functions.ipynb

In [4]:
save_files = 'files/Section1-Overview-and-Base-model'

In [5]:
save_img = 'docs/img/Section1-Overview-and-Base-model'

In [6]:
session_name = 'Section1-Overview-and-Base-model'

# Section I. Overview

## Load Saved Section if exists

In [7]:
if os.path.exists(f'sessions/{session_name}'):
    load_session(session_name)
else:
    print(f"No such session: 'sessions/{session_name}'")

## Load Data

In [8]:
data = pd.read_csv('data/train.csv')

In [9]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
data.shape

(1460, 81)

#### Check duplicates

In [11]:
data[data.duplicated()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


## Data Split

#### Train and Test datasets

In [12]:
train, test = train_test_split(
    data, test_size=0.1, random_state=seed)

In [13]:
train.shape

(1314, 81)

In [14]:
test.shape

(146, 81)

## Data Overview

#### Dataset

In [15]:
train.shape

(1314, 81)

In [ ]:
train.head()

In [ ]:
train = train.rename(columns={'SalePrice': 'price'})
test = test.rename(columns={'SalePrice': 'price'})

train.columns = map(str.lower, train.columns)

In [ ]:
columns_plot = (train
                .drop('id', axis=1)
                .select_dtypes(np.number)
                .columns.tolist())
f = plot_gridplot(
    data=train,
    features=columns_plot,
    figsize=(15,15),
    ncols=6,
    kind='hist',
    plot_shape='rectangle',
    hscale=1.6,
    histplot_kwargs={'bins': 25})

In [ ]:
train.describe()

In [ ]:
train.describe(include=object)

#### Columns rename to prevent Statsmodels errors

In [ ]:
print(sorted(train.columns.to_list(), key=str.lower))

In [ ]:
train = rename_columns(train)
test = rename_columns(test)

In [ ]:
train_raw = train.copy()

In [ ]:
print(sorted(train.columns.to_list(), key=str.lower))

#### Creating variables lists

In [ ]:
# target variable
target = 'price'

In [ ]:
ordinal = [
    'overallcond', 'overallqual', 'exterqual', 'extercond', 'bsmtqual',
    'bsmtcond', 'heatingqc', 'kitchenqual', 'fireplacequ', 'garagecond',
    'garagequal', 'poolqc', 'bsmtexposure', 'bsmtfintype_first',
    'bsmtfintype_second', 'functional', 'garagetype', 'garagefinish'
]

In [ ]:
# numeric variables
numeric = (train
           .drop(['id', 'price'], axis=1)
           .select_dtypes(include=['int64', 'float64'])
           .columns.tolist())
# remove ordinal variables from numeric list
numeric = [i for i in numeric if i not in ordinal]

In [ ]:
# categorical variables
categorical = (train
               .drop(['id', 'price'], axis=1)
               .select_dtypes(include=['object', 'bool'])
               .columns.tolist())
categorical = [i for i in categorical if i not in ordinal]

In [ ]:
# features
features = numeric + ordinal + categorical

In [ ]:
data_raw_features_len = len(data.drop('Id', axis=1).columns)
assert 0 == (data_raw_features_len
             - len(numeric)
             - len(ordinal)
             - len(categorical)
             - len([target]))

In [ ]:
len(numeric)

In [ ]:
print(numeric)

In [ ]:
len(ordinal)

In [ ]:
print(ordinal)

In [ ]:
len(categorical)

In [ ]:
print(categorical)

In [ ]:
target

#### Target Variable

In [ ]:
train[target].describe().to_frame().T

In [ ]:
train_target_raw = train[target].copy()

In [ ]:
fig = plt.figure()
sns.histplot(train_target_raw, bins=70, alpha=1)
plt.xlabel('Price')
plt.ylabel('Count')
# plt.title('Distribution of Price')
plt.grid(False)
axis_rstyle(xticks=(0, 800000, 200000), yticks=(0, 125, 25))
plt.show()

savefig('target_distribution', save_img)

In [ ]:
train[target] = np.log(train[target])

In [ ]:
fig = plt.figure()
sns.histplot(data=train, x=target, bins=50, alpha=1)
plt.xlabel('Price')
plt.ylabel('Count')
# plt.title('Distribution of Price')
plt.grid(False)
axis_rstyle(xticks=(10.0, 14.0, 0.5), yticks=(0, 120, 20))
plt.gca().tick_params(direction='out')
plt.show()

savefig('target_distribution_log', save_img)

## Missing Values Handling

### Categorical variables

#### Manually NaNs handling with data description

In [ ]:
data_describe(train[ordinal]).head()

In [ ]:
data_describe(train[categorical]).head()

##### Variables with 'NA' as NaN values (as in 'Data Description'):  
'poolqc', 'miscfeature', 'alley', 'fence', 'masvnrtype', 'fireplacequ', 'garagecond', 'garagetype', 'garagequal', 'garagefinish', 'garageyrbuilt', 'bsmtexposure', 'bsmtqual', 'bsmtcond', 'bsmtfintype1', 'bsmtfintype2'

In [ ]:
features_na = [
    'poolqc', 'miscfeature', 'alley', 'fence',
    'masvnrtype', 'fireplacequ', 'garagecond', 
    'garagetype', 'garagequal', 'garagefinish',
    'bsmtexposure', 'bsmtqual', 'bsmtcond',
    'bsmtfintype_first', 'bsmtfintype_second'
]

In [ ]:
train.head()

In [ ]:
train[features_na] = train[features_na].fillna('NA')

In [ ]:
train.head()

In [ ]:
data_describe(train[categorical]).head()

#### Categorical Imputation

Most frequent. <br>
The only NaN value of 'electrical' variable is filled by the most frequent value

In [ ]:
imputer_frequent = CategoricalImputer(
    imputation_method='frequent',
    variables=categorical
)

In [ ]:
imputer_frequent.fit(train)

In [ ]:
train = imputer_frequent.transform(train)

In [ ]:
data_describe(train[categorical]).head(5)

In [ ]:
train[categorical].isna().values.any()

### Quantitative variables

In [ ]:
data_describe(train[numeric]).head(5)

#### 'garageyrblt' 

Filling NaN values by 'yearbuilt' value

In [ ]:
sns.histplot(train['garageyrblt'], bins=50);

In [ ]:
garage_fill = train.loc[train['garageyrblt'].isna(), 'yearbuilt']

In [ ]:
loc = train['garageyrblt'].isna(), 'garageyrblt'
train.loc[loc] = train.loc[loc].fillna(garage_fill)

In [ ]:
sns.histplot(
    data=train, x='garageyrblt',
    hue='garagetype', bins=50
);
plt.legend(
    labels = train['garagetype'].unique(),
    loc='upper right', bbox_to_anchor=(1.2, 1),
    prop={'size': 9}, labelcolor='0.3' 
)
plt.show()

#### 'lotfrontage'

1. Check is there dependency between 'lotfrontage'==NaN and target

**Results:** 
1. no dependency **-->** mean imputation
2. Additional: detected two potential outliers (idxs: 934, 1298)

In [ ]:
train['lotfrontage'].hist(bins=50);

In [ ]:
(train[numeric]
 .corr()
 .loc['lotfrontage']
 .sort_values(ascending=False, key=abs)
 .head())

In [ ]:
# lotfrontage NaNs range in 'price' variable
fig = plt.figure()
sns.scatterplot(
    data=train,
    x='id',
    y=target,
    color=palette[0]
)
loc_r = train['lotfrontage'].isna()
sns.scatterplot(
    data=train.loc[loc_r],
    y='price',
    x=train.loc[loc_r, 'id'].values,
    color=palette[1]
)
plt.xlabel(str.capitalize('id'))
plt.ylabel(str.capitalize('price'))
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 2.5))

sns.scatterplot(
    data=train, x='lotfrontage', y=target, **scatter)

sns.scatterplot(
    data=train.loc[train['lotfrontage']>250, ['lotfrontage', target]],
    x='lotfrontage',
    y=target,
    s=20,
    color=palette[1])

plt.xlabel('Lotfrontage')
plt.ylabel('Price')
# plt.title('Street connected to property vs Price')
axis_rstyle(
    yticks=[10, 14, 0.5],
    xticks=[0, 350, 50],
    grid=False
)
plt.show()

savefig('lotfrontage_outs', save_img)

In [ ]:
train[train['lotfrontage']>250]

#### 'masvnrarea'

Potential outliers (idx: 297)

In [ ]:
masvnrarea_nan_idxs = train[train['masvnrarea'].isna()].index.tolist()

In [ ]:
masvnrarea_nan_idxs

In [ ]:
train.loc[masvnrarea_nan_idxs, ['masvnrarea', 'masvnrtype', 'price']]

In [ ]:
sns.scatterplot(data=train, x='masvnrarea', y=target);

In [ ]:
train[train['masvnrarea'] > 1400]

In [ ]:
train.loc[train['masvnrarea'].isna(), 'masvnrarea'] = 0

#### Quantitative Imputation 

Median

In [ ]:
imputer_median = MeanMedianImputer(
    imputation_method='median',
    variables=numeric
)

In [ ]:
imputer_median.fit(train)

In [ ]:
train = imputer_median.transform(train)

In [ ]:
train.head()

In [ ]:
data_describe(train[numeric]).head()

In [ ]:
train[numeric].isna().values.any()

## Relabeling 

In [ ]:
features_labels_lists = [
    ['exterqual', 'extercond', 'bsmtqual', 'bsmtcond', 'heatingqc',
     'kitchenqual', 'fireplacequ', 'garagecond', 'garagequal', 'poolqc'],
    ['bsmtexposure'],
    ['bsmtfintype_first', 'bsmtfintype_second'],
    ['functional'],
    ['garagetype'],
    ['garagefinish'],
    ['mssubclass']
]

In [ ]:
features_labels_dicts = [
    {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0},
    {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0},
    {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0},
    {'Typ': 7, 'Min1': 6, 'Min2': 5, 'Mod': 4, 'Maj1': 3, 'Maj2': 2,
     'Sev': 1, 'Sal': 0},
    {'2Types': 6, 'Attchd': 5, 'Basment': 4, 'BuiltIn': 3,
     'CarPort': 2, 'Detchd': 1, 'NA': 0},
    {'Fin': 3, 'RFn': 2, 'Unf': 1, 'NA': 0}
]

In [ ]:
train['exterqual'].unique()

In [ ]:
train['poolqc'].unique()

In [ ]:
train['mssubclass'].unique()

In [ ]:
zip_ = zip(features_labels_lists, features_labels_dicts)
for lst, dct in zip_:
    for f in lst:
        train[f] = train[f].map(dct)

In [ ]:
train['mssubclass'] = train['mssubclass'].apply(lambda x: str(x))

In [ ]:
train['exterqual'].unique()

In [ ]:
train['poolqc'].unique()

In [ ]:
train['mssubclass'].unique()

### Transform 'msubclass' values and move it to Categorical variables list

In [ ]:
categorical.append('mssubclass')
numeric.remove('mssubclass')

In [ ]:
assert len(features) == len(numeric) + len(ordinal) + len(categorical)

In [ ]:
categorical_transform = categorical.copy()

## Transformation Pipeline Check

In [ ]:
transformer_overview = OverviewTransformer(
    features_na=features_na
)

In [ ]:
transformer_label = LabelTransformer(
    features_labels_lists=features_labels_lists,
    features_labels_dicts=features_labels_dicts
)

In [ ]:
pipeline = Pipeline(steps=[
    ('overview', transformer_overview),
    ('imputer_num', imputer_median),
    ('imputer_cat', imputer_frequent),
    ('label', transformer_label)
])

In [ ]:
pipeline

In [ ]:
train_transformed = train_raw.copy()

In [ ]:
train_transformed = pipeline.transform(train_transformed)

In [ ]:
train_transformed[target] = np.log(train_transformed[target])

In [ ]:
# check pipeline - datasets have to be equals if pipeline if correct
assert train_transformed.equals(train)

In [ ]:
# display rows with differences
train[~train.apply(tuple, 1).isin(train_transformed.apply(tuple, 1))]

## Baseline Model

In [ ]:
train_bm = train.copy()

In [ ]:
train_bm.head()

### Pipeline For GridSearch

In [ ]:
encoder = OrdinalEncoder(
    encoding_method='ordered',
    variables=categorical_transform,
    missing_values='ignore',
    unseen='encode'
)

In [ ]:
lgb1 = LGBMRegressor()

In [ ]:
lgb_pipeline1 = Pipeline(steps=[
    ('encoder', encoder),
    ('estimator', lgb1)
])

### GridSearch Iteration #1

In [ ]:
params1 = {
    'estimator__n_estimators': [25, 50, 75],
    'estimator__max_depth': [3, 6, 9],
    'estimator__num_leaves': [5, 38, 300],
    'estimator__learning_rate': [0.1, 0.3],
    'estimator__random_state': [seed],
    'estimator__verbose': [-1]
}

In [ ]:
n_folds1 = 20

In [ ]:
cv1 = GridSearchCV(
    estimator=lgb_pipeline1, 
    param_grid=params1,
    scoring='neg_root_mean_squared_error',
    cv=n_folds1
)

In [ ]:
st = stopwatch.start()
cv1.fit(train_bm[features], train_bm[target])
print(f'Execution time: {stopwatch.stop(st)}')

In [ ]:
cv1_results_df = pd.DataFrame(cv1.cv_results_)

In [ ]:
cv1_results_df.shape

In [ ]:
cv1_results_df_columns_drop = [
    'mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
]

In [ ]:
cv1_results_df = cv1_results_df.loc[
    :, ~cv1_results_df.columns.isin(cv1_results_df_columns_drop)].copy()

In [ ]:
cv1_results_df = cv1_results_df.sort_values([
    'param_estimator__learning_rate', 'param_estimator__n_estimators',
    'param_estimator__max_depth', 'param_estimator__num_leaves'
], ascending=True)

In [ ]:
cv1_results_df = cv1_results_df.reset_index(drop=True)

In [ ]:
# folds scores are normal
test_normality(cv1_results_df.iloc[0, 7:27].astype(float))

In [ ]:
cv1_results_df['ci_margin'] = \
    1.96 * cv1_results_df['std_test_score']  / np.sqrt(n_folds1)

In [ ]:
cv1_results_df_params_cols = [
    'param_estimator__learning_rate', 'param_estimator__n_estimators',
    'param_estimator__max_depth', 'param_estimator__num_leaves'
]

In [ ]:
cv1_results_df.head()

In [ ]:
# sort thw simplest model if several equal mean_test_scores
cv1_results_df_best_score_idx = \
    (cv1_results_df
     .sort_values(['param_estimator__n_estimators',
                   'param_estimator__max_depth',
                   'param_estimator__num_leaves'])
     .sort_values(['mean_test_score'], ascending=False)).iloc[0].name

In [ ]:
cv1_results_df_best_score_idx

In [ ]:
cv1_results_df_best_score = cv1_results_df.loc[
    cv1_results_df.index==cv1_results_df_best_score_idx, 'mean_test_score'
].item()

In [ ]:
cv1_results_df_best_score_ci_margin = cv1_results_df.loc[
    cv1_results_df.index==cv1_results_df_best_score_idx, 'ci_margin'
].item()

In [ ]:
# define colors of pointplot's elements
cv1_results_df_colors = [palette[-4]] * 54
cv1_results_df_colors[cv1_results_df_best_score_idx] = '#B63125'

In [ ]:
fig, ax = plt.subplots(figsize=(10, 2.5))

zip_ = [
    cv1_results_df.index, cv1_results_df['mean_test_score'],
    cv1_results_df['ci_margin'], cv1_results_df_colors
]
for x_, y_, ci_, c_ in zip(*zip_): 
    if x_ == cv1_results_df_best_score_idx:
        ax.errorbar(
            x=x_,
            y=y_,
            yerr=ci_,
            fmt='o',
            markersize=2,
            c=c_,
            linestyle='none',
            elinewidth=1,
            capsize=1.1,
            ecolor=c_,
            zorder=1,
        )
    else:
        ax.errorbar(
            x=x_,
            y=y_,
            yerr=ci_,
            fmt='o',
            markersize=1.5,
            c=c_,
            linestyle='none',
            elinewidth=1,
            capsize=1,
            ecolor=c_,
            zorder=1
        )
ax.set_xticks(ticks=arange(len(cv1_results_df)), labels=['5', '38', '300'] * 18)
axis_rstyle()

axis_add_xaxis(
    labels=[
        ['Dep.: 3', 'Dep.: 6', 'Dep.: 9'] * 6,
        ['Trees: 25', 'Trees: 50', 'Trees:75'] * 2,
        ['Learning rate: 0.1', 'Learning rate: 0.3']
    ],
    labelcolor='#606060'
)
savefig('gridsearch1', save_img)

In [ ]:
best_params1 = cv1_results_df.loc[
    cv1_results_df.index==cv1_results_df_best_score_idx, 'params'
].item()

In [ ]:
best_score1 = round(cv1_results_df_best_score, 4)

In [ ]:
best_score1

In [ ]:
best_params1

### GridSearch Iteration #2

In [ ]:
lgb2 = LGBMRegressor()

In [ ]:
lgb_pipeline2 = Pipeline(steps=[
    ('encoder', encoder),
    ('estimator', lgb2)
])

In [ ]:
learning_rate = [best_params1['estimator__learning_rate']]

max_depth_val = best_params1['estimator__max_depth']
max_depth = list(arange(max_depth_val-2, max_depth_val+3))

n_estimators_val = best_params1['estimator__n_estimators']
n_estimators = [n_estimators_val-5, n_estimators_val, n_estimators_val+5]

num_leaves_val = best_params1['estimator__num_leaves']
num_leaves = [num_leaves_val-15, num_leaves_val, num_leaves_val+15]

In [ ]:
params2 = {
    'estimator__learning_rate': learning_rate,
    'estimator__max_depth': max_depth,
    'estimator__n_estimators': n_estimators,
    'estimator__num_leaves': num_leaves,
    'estimator__random_state': [seed],
    'estimator__verbose': [-1]
}

In [ ]:
params2

In [ ]:
n_folds2 = 20

In [ ]:
cv2 = GridSearchCV(
    estimator=lgb_pipeline2, 
    param_grid=params2,
    scoring='neg_root_mean_squared_error',
    cv=n_folds2
)

In [ ]:
st = stopwatch.start()
cv2.fit(train_bm[features], train_bm[target])
print(f'Execution time: {stopwatch.stop(st)}')

In [ ]:
cv2_results_df = pd.DataFrame(cv2.cv_results_)

In [ ]:
cv2_results_df.shape

In [ ]:
cv2_results_df_columns_drop = [
    'mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
]

In [ ]:
cv2_results_df = cv2_results_df.loc[
    :, ~cv2_results_df.columns.isin(cv2_results_df_columns_drop)].copy()

In [ ]:
cv2_results_df = cv2_results_df.sort_values([
    'param_estimator__learning_rate', 'param_estimator__n_estimators',
    'param_estimator__max_depth', 'param_estimator__num_leaves'
], ascending=True)

In [ ]:
cv2_results_df = cv2_results_df.reset_index(drop=True)

In [ ]:
# folds scores are normal
test_normality(cv2_results_df.iloc[0, 7:27].astype(float))

In [ ]:
cv2_results_df['ci_margin'] = \
    1.96 * cv2_results_df['std_test_score'] / np.sqrt(n_folds1)

In [ ]:
cv2_results_df_params_cols = [
    'param_estimator__learning_rate', 'param_estimator__n_estimators',
    'param_estimator__max_depth', 'param_estimator__num_leaves'
]

In [ ]:
cv2_results_df.head()

In [ ]:
# sort thw simplest model if several equal mean_test_scores
cv2_results_df_best_score_idx = \
    (cv2_results_df
     .sort_values(['param_estimator__n_estimators',
                   'param_estimator__max_depth',
                   'param_estimator__num_leaves'])
     .sort_values(['mean_test_score'], ascending=False)).iloc[0].name

In [ ]:
cv2_results_df_best_score_idx

In [ ]:
cv2_results_df_best_score = cv2_results_df.loc[
    cv2_results_df.index==cv2_results_df_best_score_idx, 'mean_test_score'
].item()

In [ ]:
cv2_results_df_best_score_ci_margin = cv2_results_df.loc[
    cv2_results_df.index==cv2_results_df_best_score_idx, 'ci_margin'
].item()

In [ ]:
len(cv2_results_df)

In [ ]:
# define colors of pointplot's elements
cv2_results_df_colors = [palette[-4]] * 45
cv2_results_df_colors[cv2_results_df_best_score_idx] = '#B63125'

In [ ]:
fig, ax = plt.subplots(figsize=(10, 2.5))

zip_ = [
    cv2_results_df.index, cv2_results_df['mean_test_score'],
    cv2_results_df['ci_margin'], cv2_results_df_colors
]
for x_, y_, ci_, c_ in zip(*zip_): 
    if x_ == cv2_results_df_best_score_idx:
        ax.errorbar(
            x=x_,
            y=y_,
            yerr=ci_,
            fmt='o',
            markersize=2,
            c=c_,
            linestyle='none',
            elinewidth=1,
            capsize=1.1,
            ecolor=c_,
            zorder=1,
        )
    else:
        ax.errorbar(
            x=x_,
            y=y_,
            yerr=ci_,
            fmt='o',
            markersize=1.5,
            c=c_,
            linestyle='none',
            elinewidth=1,
            capsize=1,
            ecolor=c_,
            zorder=1
        )
ax.set_xticks(ticks=arange(len(cv2_results_df)), labels=['285', '300', '315']*15)
axis_rstyle(ylim=[-0.160, -0.099], yticks=[-0.160, -0.099, 0.020])

axis_add_xaxis(
    labels=[
        ['Dep.: 7', 'Dep.: 8', 'Dep.: 9', 'Dep.: 10', 'Dep.: 11'] * 3,
        ['Trees: 70', 'Trees: 75', 'Trees: 80'],
        ['Learning rate: 0.1']
    ],
    labelcolor='#606060'
)

plt.show()

savefig('gridsearch2', save_img)

In [ ]:
params_opt = cv2_results_df.loc[
    cv2_results_df.index==cv2_results_df_best_score_idx, 'params'
].item()

In [ ]:
rmse_train_opt = round(cv2_results_df_best_score, 4)

In [ ]:
rmse_train_opt

In [ ]:
params_opt

In [ ]:
params = {}
for key in params_opt.keys():
    new_key = key.lstrip('estimator')
    new_key = new_key.lstrip('__')
    params[new_key] = params_opt[key]

In [ ]:
params

In [ ]:
lgb_opt = LGBMRegressor(**params)

In [ ]:
train_bm_opt = train_bm.copy()

In [ ]:
train_bm_opt[features] = encoder.fit_transform(
    train_bm_opt[features], train_bm_opt[target])

In [ ]:
train_bm_opt.head()

In [ ]:
lgb_opt.fit(train_bm_opt[features], train_bm_opt[target])

In [ ]:
feature_importance = pd.DataFrame({
    'Feature': lgb_opt.feature_name_,
    'Importance': lgb_opt.feature_importances_
})
feature_importance.sort_values('Importance', ascending=False, inplace=True)

In [ ]:
data_plot = feature_importance.loc[feature_importance['Importance']>0,:]
data_plot = data_plot.sort_values('Importance', ascending=False)

In [ ]:
data_plot.head()

In [ ]:
values = data_plot['Importance'][:10]
labels = data_plot['Feature'][:10]

In [ ]:
fig = plot_bar_horizontal(
    values=values,
    labels=labels,
    labelsize=9,
    labelcolor='0.3',
    labelweight='medium',
    figsize=(3, 2.5),
    kind='lol',
    width=0.5,
    s=6,
    x_lim_right=250
)
savefig('feature_importance', save_img)

## Save Data

In [ ]:
datasets_dict = {
    'train_raw': train_raw,
    'train': train,
    'test': test
}

In [ ]:
features_dict = {
    'features': features,
    'numeric': numeric,
    'ordinal': ordinal,
    'categorical': categorical,
    'categorical_transform': categorical_transform,
    'target': target
}

In [ ]:
transform_dict = {
    'features_na': features_na,
    'transformer_overview': transformer_overview,
    'transformer_imputer_frequent': imputer_frequent,
    'transformer_imputer_median': imputer_median,
    'transformer_label': transformer_label,
}

In [ ]:
simulation_datasets_dict = {
    'train_base': train_bm,
    'features_base': features
}

In [ ]:
estimators_dict = {'lgb_base': lgb_opt}

In [ ]:
evaluation_dict = {'cv_base': cv2}

In [ ]:
saveit(datasets_dict, 'datasets_dict', save_files)

In [ ]:
saveit(features_dict, 'features_dict', save_files)

In [ ]:
saveit(transform_dict, 'transform_dict', save_files)

In [ ]:
saveit(simulation_datasets_dict, 'simulation_datasets_dict', save_files)

In [ ]:
saveit(estimators_dict, 'estimators_dict', save_files)

In [ ]:
saveit(evaluation_dict, 'evaluation_dict', save_files)

In [ ]:
saveit(params, 'params', save_files)

In [ ]:
saveit(feature_importance, 'feature_importance', save_files)

### Save Session

In [ ]:
save_session(session_name)

### Execution time

In [ ]:
print(f'Execution time: {stopwatch.stop(start)}')